In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import io 

In [3]:
# import file

from google.colab import files
uploaded = files.upload()

Saving sample_submission.csv to sample_submission.csv


In [3]:
dataset = pd.read_csv('data.csv',parse_dates=True)

In [4]:
dataset

,Date,Open,High,Low,Close,Volume
0,2010-09-13,15400,15519,15320,15400,234851
1,2010-09-14,15500,15620,15200,15200,371530
2,2010-09-15,15260,15340,15020,15140,389289
3,2010-09-16,15080,15260,15080,15140,234815
4,2010-09-17,15240,15560,15159,15460,447649
...,...,...,...,...,...,...
1623,2017-04-06,42000,42080,41600,41840,193668
1624,2017-04-07,41800,41820,41160,41600,200420
1625,2017-04-10,41940,41940,41500,41940,142827
1626,2017-04-11,41940,41940,41580,41600,143458


In [5]:
# seeing the data

dataset.describe()

,Open,High,Low,Close,Volume
count,1628.000000,1628.000000,1628.000000,1628.000000,1.628000e+03
mean,25557.827396,25813.232187,25303.552211,25564.656020,2.880353e+05
std,5412.200956,5457.414669,5394.723398,5436.660567,1.355355e+05
min,13680.000000,13940.000000,13440.000000,13600.000000,6.598500e+04
25%,22690.000000,22970.000000,22480.000000,22700.000000,1.991875e+05
50%,25900.000000,26080.000000,25670.000000,25900.000000,2.534460e+05
75%,28545.000000,28844.750000,28300.000000,28545.000000,3.364635e+05
max,42200.000000,42680.000000,41880.000000,42560.000000,1.277983e+06


In [6]:
dataset.info()
#no null values no zero values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1628 entries, 0 to 1627
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    1628 non-null   object
 1   Open    1628 non-null   int64 
 2   High    1628 non-null   int64 
 3   Low     1628 non-null   int64 
 4   Close   1628 non-null   int64 
 5   Volume  1628 non-null   int64 
dtypes: int64(5), object(1)
memory usage: 76.4+ KB


In [ ]:
#observe the plot 

dataset.plot(x="Date",y="Close",figsize=(16,8),title="Stonks",label="Closing price")

In [8]:
# just to double confm no null values

dataset.isnull().values.any()

False

In [9]:
train_data, test_data = dataset[0:int(len(dataset)*0.8)], dataset[int(len(dataset)*0.8):]

In [10]:
# Check if price series is stationary
from statsmodels.tsa.stattools import adfuller

result = adfuller(train_data.Close)
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")

#it is not staionary as p value is more 0.05 so nd to differentiate

ADF Statistic: -2.204728526466629
p-value: 0.20453375302022486


In [7]:
# find d term suing adf test

!pip install pmdarima scipy==1.2-Uqq


from pmdarima.arima.utils import ndiffs
ndiffs(train_data.Close, test="adf")

#d = 1

ERROR: Could not find a version that satisfies the requirement scipy==1.2-Uqq (from versions: 0.8.0, 0.9.0, 0.10.0, 0.10.1, 0.11.0, 0.12.0, 0.12.1, 0.13.0, 0.13.1, 0.13.2, 0.13.3, 0.14.0, 0.14.1, 0.15.0, 0.15.1, 0.16.0, 0.16.1, 0.17.0, 0.17.1, 0.18.0, 0.18.1, 0.19.0, 0.19.1, 1.0.0b1, 1.0.0rc1, 1.0.0rc2, 1.0.0, 1.0.1, 1.1.0rc1, 1.1.0, 1.2.0rc1, 1.2.0rc2, 1.2.0, 1.2.1, 1.2.2, 1.2.3, 1.3.0rc1, 1.3.0rc2, 1.3.0, 1.3.1, 1.3.2, 1.3.3, 1.4.0rc1, 1.4.0rc2, 1.4.0, 1.4.1, 1.5.0rc1, 1.5.0rc2, 1.5.0, 1.5.1, 1.5.2, 1.5.3, 1.5.4, 1.6.0rc1, 1.6.0rc2, 1.6.0, 1.6.1, 1.6.2, 1.6.3, 1.7.0rc1, 1.7.0rc2, 1.7.0, 1.7.1)
ERROR: No matching distribution found for scipy==1.2-Uqq


1

In [ ]:
#inspect autocorrealtion plot 

from statsmodels.graphics.tsaplots import plot_pacf

diff = train_data.Close.diff().dropna()





plot_pacf(diff)
#p = 1 iwanted to choose 23 as it was above significance level but 1 is close too

In [ ]:
#inspect autocorrealtion plot
from statsmodels.graphics.tsaplots import plot_acf

plot_acf(train_data.Close.diff().dropna())


#q = 1 , iwanted to choose 23 as it was above significance level but 1 is close too

In [ ]:

from statsmodels.tsa.arima_model import ARIMA
import warnings


warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

# ARIMA Model
model = ARIMA(train_data["Close"], order=(1, 1, 1))
result = model.fit()

In [ ]:
result.summary()

In [ ]:
start=len(train_data)
end=len(train_data)+len(test_data)-1
pred=result.predict(start=start,end=end,typ='levels').rename('ARIMA Predictions')
pred.plot(legend=True)
test_data['Close'].plot(legend=True)

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
test_data['Close'].mean()
rmse=sqrt(mean_squared_error(pred,test_data['Close']))
print(rmse)

In [ ]:
print(pred)

In [ ]:
test_data['Close'].mean()

In [ ]:
pred.values

In [ ]:
MAmodel = ARIMA(train_data.Close, order=(0, 1, 1))
MAresult = MAmodel.fit(disp=0)

In [ ]:
MAresult.summary()

In [ ]:
pred=MAresult.predict(start=start,end=end,typ='levels').rename('AMA Predictions')
pred.plot(legend=True)
test_data['Close'].plot(legend=True)

In [ ]:
rmse=sqrt(mean_squared_error(pred,test_data['Close']))
print(rmse)
#rmse for MA prediction

In [ ]:
ARmodel = ARIMA(train_data.Close, order=(1, 1, 0))
ARresult = ARmodel.fit(disp=0)

In [ ]:
pred=ARresult.predict(start=start,end=end,typ='levels').rename('AR Predictions')
pred.plot(legend=True)
test_data['Close'].plot(legend=True)

In [ ]:
rmse=sqrt(mean_squared_error(pred,test_data['Close']))
print(rmse)
#rmse for AR prediction

In [ ]:
model1 = ARIMA(dataset.Close, order=(1, 1, 1))
result1 = model1.fit(disp=0)

#choose arima model as lowest rmse


In [ ]:
start=len(train_data)

end=len(train_data)+len(test_data)-1
pred=result1.predict(start=start,end=end,typ='levels').rename('ARIMA Predictions')
pred.plot(legend=True)

test_data['Close'].plot(legend=True)

In [ ]:
result1.summary()

In [ ]:
#for 1086 rows from the end of the train data we have 

pred = result1.predict(start=len(dataset),end=len(dataset)+1085,typ="levels")

In [ ]:
ans = pd.read_csv('sample_submission.csv',parse_dates=True)

In [ ]:
ans['Predicted'] = pred.values

In [ ]:
ans

In [ ]:
ans.to_csv("ans.csv", sep='\t',index=False)

In [ ]:
from google.colab import drive
drive.mount('drive')

!cp ans.csv "drive/My Drive/"

In [ ]:
!cp ans.csv "drive/My Drive/"

In [ ]:
"""model2 = ARIMA(dataset.Close, order=(7, 1, 7))
result2 = model2.fit(disp=0)
#index_future_dates=pd.date_range(start='2017-04-13',end='2021-09-13')
#print(index_future_dates)
pred=result2.predict(start=len(dataset),end=len(dataset)+1085).rename('ARIMA Predictions')
#print(comp_pred)

print(pred)"""
